In [1]:
from tqdm.auto import tqdm

import shutil
import os


def load_img_path(load_data_path: str, img_path: str, labels_path: str) -> tuple:
    img_paths, labels_paths = [], []
    img, lbl = os.listdir(img_path), os.listdir(labels_path)
    img.sort(); lbl.sort()

    with open(load_data_path, 'r') as f:
        for name in f:
            name = name.strip()
            name = name.replace('\n', '') + '.jpg'
            if name in img: img_paths.append(f'{img_path}/{name}')
            else: continue

            name = name.replace(".jpg", ".txt")
            if name in lbl: labels_paths.append(f'{labels_path}/{name}')
            else: img_paths.remove(f'{img_path}/{name}') # Fail safe
    
    return (img_paths, labels_paths)


def process_data(mode: str, load_data_list: tuple, save_path: str) -> None:
    img, label = load_img_path(*load_data_list)

    output_paths = [save_path + f'/{mode}/images', save_path + f'/{mode}/labels']
    for _path in output_paths:
        if not os.path.isdir(_path):
            os.makedirs(_path)
    
    for i in tqdm(range(len(img))):
        image_path = img[i]
        label_path = label[i]

        shutil.copy(image_path, output_paths[0])
        shutil.copy(label_path, output_paths[1])


data_path = '../input/diordata'
save_path = '/kaggle/temp/Model_Dataset'

train_img_name_path = data_path + '/ImageSets/train.txt'
val_img_name_path = data_path + '/ImageSets/val.txt'
test_img_name_path = data_path + '/ImageSets/test.txt'

img_path = data_path + '/images'
labels_path = data_path + '/labels'

#Train
process_data('train', (train_img_name_path, img_path, labels_path), save_path)
# Val
process_data('val', (val_img_name_path, img_path, labels_path), save_path)
# Test
process_data('test', (test_img_name_path, img_path, labels_path), save_path)

  0%|          | 0/19004 [00:00<?, ?it/s]

  0%|          | 0/2112 [00:00<?, ?it/s]

  0%|          | 0/2347 [00:00<?, ?it/s]

In [3]:
!pip install ultralytics
!pip install albumentations==1.0.3

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/f4/79/b192c64e13eefc7f8543a10080767ab057850fa9bdbc38e7c261a3432c93/ultralytics-8.0.227-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.5/660.5 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.3.1
    Uninstalling albumentations-1.3.1:
      Successfully uninstalled albumentations-1.3.1


In [4]:
from ultralytics import YOLO

import yaml

model_data_path = '/kaggle/working/Model'

model = YOLO('/kaggle/working/runs/detect/Yolov8/weights/best.pt')
labels = [
    'Storage Tank',
    'Baseball field',
    'Tennis court',
    'Basketball Court',
    'Wind mill',
    'Vehicle',
    'Harbor',
    'Ship',
    'Airplane',
    'Bridge',
    'Overpass',
    'Expressway toll station',
    'Train station',
    'Chimney',
    'Ground Track Field',
    'Dam',
    'Expressway service area',
    'Stadium',
    'Airport',
    'Golf course'
]
config = {
    'path': '/kaggle/temp',
    'train': 'Model_Dataset/train/images',
    'val': 'Model_Dataset/val/images',
    'test': 'Model_Dataset/test/images',
    'names': {str(i): label for i, label in enumerate(labels)}
}

with open(model_data_path + '\config.yaml', 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

model.train(data=model_data_path + '\config.yaml', imgsz=800, epochs=20, batch=16, name='Yolov8_1')

Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/runs/detect/Yolov8/weights/best.pt, data=/kaggle/working/Model\config.yaml, epochs=20, patience=50, batch=16, imgsz=800, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=Yolov8_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

100%|██████████| 755k/755k [00:00<00:00, 22.2MB/s]
2023-12-15 11:03:21,737	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-12-15 11:03:22,653	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


...

                   all       2112      17972      0.868      0.769      0.831      0.591

20 epochs completed in 2.096 hours.
Optimizer stripped from runs/detect/Yolov8_1/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/Yolov8_1/weights/best.pt, 6.3MB

Validating runs/detect/Yolov8_1/weights/best.pt...
Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)
YOLOv8n summary (fused): 168 layers, 3009548 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 66/66 [00:37<00:00,  1.78it/s]


                   all       2112      17972      0.867      0.769      0.831      0.591
          Storage Tank       2112       2824      0.915      0.746       0.84      0.525
        Baseball field       2112        449      0.947      0.949      0.974      0.834
          Tennis court       2112       1085      0.959      0.933      0.965      0.835
      Basketball Court       2112        294      0.929      0.874      0.927        0.8
             Wind mill       2112        500      0.928      0.836      0.908      0.475
               Vehicle       2112       4017       0.87      0.492      0.657      0.364
                Harbor       2112        421      0.747      0.646      0.711      0.503
                  Ship       2112       5556      0.931      0.927      0.954      0.589
              Airplane       2112        912       0.96       0.94      0.969      0.704
                Bridge       2112        376       0.85      0.428      0.565      0.311
              Overpas

lr/pg0,▃▆███▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
lr/pg1,▃▆███▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
lr/pg2,▃▆███▇▇▆▆▅▅▅▄▄▃▃▂▂▁▁
metrics/mAP50(B),▃▁▁▂▂▂▃▄▃▃▅▅▆▆▆▇▇▇▇█
metrics/mAP50-95(B),▃▂▁▁▂▂▂▃▄▃▄▅▆▆▇▇▇▇██
metrics/precision(B),▃▁▃▃▂▂▅▅▄▁▇▆▆▇▇▇▆▆▇█
metrics/recall(B),▄▁▁▁▁▁▂▅▄▆▃▅█▅▅▆▇███
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▆▇████▇▇▆▆▄▄▃▃▃▂▂▂▁▁


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7862c1c62a10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043